In [10]:
!pip install -r ../requirements.txt

In [12]:
import os
import json
import requests

import sqlite3
from tqdm.notebook import tqdm, trange
tqdm.pandas()

import numpy as np
import pandas as pd 
from sqlalchemy import create_engine

%load_ext sql
%config SqlMagic.autocommit=True

from tqdm.notebook import tqdm
from pprint import pprint as print

ImportError: cannot import name 'escape_string_literals_with_colon_prefix' from 'sql.parse' (/Users/jessicahyne/opt/miniconda3/envs/gymternet/lib/python3.11/site-packages/sql/parse.py)

In [ ]:
-- %load_ext sql
-- %config SqlMagic.autocommit=True

ImportError: cannot import name 'escape_string_literals_with_colon_prefix' from 'sql.parse' (/Users/jessicahyne/opt/miniconda3/envs/gymternet/lib/python3.11/site-packages/sql/parse.py)

# 1 Clean up `teams_df` and create `Teams.db`

What I want in this table in my database is:

| **team_id** | **team_name** | **team_url**                           |
|-------------|---------------|----------------------------------------|
| 1           | 'Auburn'      | "https://www.roadtonationals/team/645" | 


etc.

The `team_id` column should act as the primary key.


In [ ]:
# Make a database to keep all my lovely tables in
%sql sqlite:///../Data/Clean/gymternet.db --alias gymternet
engine = create_engine('sqlite:///../Data/Clean/gymternet.db')

In [ ]:
teams_df = pd.read_pickle('../Data/Raw/dirty_dfs/teams_df.pkl')
meets_df = pd.read_pickle('../Data/Raw/dirty_dfs/meets_df.pkl')
gymnasts_data_df = pd.read_pickle('../Data/Raw/dirty_dfs/gymnasts_data_df.pkl')
team_results_data_df = pd.read_pickle('../Data/Raw/dirty_dfs/team_results_data_df.pkl')

# 1 Clean up `teams_df` and create `Teams.db`

What I want in this table in my database is:

| **team_id** | **team_name** | **team_url**                           |
|-------------|---------------|----------------------------------------|
| 1           | 'Auburn'      | "https://www.roadtonationals/team/645" | 


etc.

The `team_id` column should act as the primary key.


In [ ]:
# Preview the dataframe
#teams_df.dtypes
teams_df.head()

This dataframe is already pretty tidy, but the links are pointing towards the API, which is not what we want, and we still have an irrelevant column (`year`)

In [ ]:
base_team_url = 'https://roadtonationals.com/results/teams/dashboard'
teams_df['team_url'] = teams_df.apply(lambda x: f'{str(base_team_url)}/{str(x["year"])}/{str(x["team_id"])}', axis=1)

In [ ]:
# Preview the df
teams_df.head()

In [ ]:
# Drop the year column
teams_df.drop(columns='year', inplace=True)
teams_df.head()

In [ ]:
teams_df.to_sql('teams', con=engine, if_exists = 'append', chunksize = 1000)

In [ ]:
meets_df.head()

# 1 Clean up `meets_df` and create `meets.db`

What I want in this table in my database is:

| **meet_id** | **year**      | **team_id** | **date**   |
|-------------|---------------|-------------|------------|
| 117897      | 2015          | 1           | 2015-01-09 |


etc.

The `meet_id` column should act as the primary key, the `team_id` column is a foreign key, connecting to the `teams` table.

This DataFrame is a bit more of a mess, so will require a bit more cleaning before I send to the DB.

In [ ]:
meets_df = pd.read_pickle('../Data/Raw/dirty_dfs/meets_df.pkl')
# Preview the meets_df
meets_df.head()

In [ ]:
# Drop the irrelevant columns: team_name, team_id, team_score, opponent, linked_id, jas)
meets_df.drop(columns=['team_name', 'team_id', 'home', 'meet_desc', 'team_score', 'opponent', 'linked_id', 'jas'], inplace=True)

# Preview the meets_df
meets_df.head()

In [ ]:
# Tidy up the data types
meets_df['meet_id'] = meets_df['meet_id'].astype(int)

# Change the format of the meet_date column to be compatible with datetime
meets_df['meet_date'] = meets_df['meet_date'].apply(lambda x: x.split(' ')[1])

# Change the "Jan", "Feb", etc. to numbers
months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
meets_df['meet_date'] = meets_df['meet_date'].apply(lambda x: f'{x.split("-")[2]}-{months[x.split("-")[0]]}-{x.split("-")[1]}')

# Change the datatype of the meet_date column to datetime
meets_df['meet_date'] = pd.to_datetime(meets_df['meet_date'])

# Tidy up the meet_url column
base_meet_url = 'https://roadtonationals.com/results/schedule/meet/'
meets_df['meet_url'] = meets_df.apply(lambda x: f'{str(base_meet_url)}{str(x["meet_id"])}', axis=1)

# Preview the meets_df
meets_df.dtypes


In [ ]:
# Melt the all_teams column so that each of the teams in the list has its own row
meets_df = meets_df.explode('all_teams').reset_index(drop=True)

# Preview the meets_df
meets_df.head()

In [ ]:
# Merge the meets_df with the teams_df to get the team_id
meets_df = meets_df.merge(teams_df[['team_id', 'team_name']], left_on='all_teams', right_on='team_name', how='left')

# Drop the team_name column
meets_df.drop(columns=['team_name', 'all_teams'], inplace=True)

# Rename the team_id column
meets_df.rename(columns={'team_id': 'team_id'}, inplace=True)

# Preview the meets_df
meets_df.head()


In [ ]:
# Find the NA values in the team_id column
meets_df[meets_df['team_id'].isna()]

Looking at the dataframe, I noticed there are some NA values in the team_id column (indicating a meet with only one team participating). This seemed curious (as typically a competition requires at least two parties), and upon exploring the data, it seems there is a coding error on the website or perhaps there are meets listed that did not end up taking place due to cancellations, but were still listed as 'intended meets'. Furthermore, some teams are listed as participants for (semi) finals that did not actually make it through to that round of competition. 

As such, I can feel safe dropping dropping those meet_ids from the df.

Incidentally, these are the same meet_ids that weren't able to be retrieved in the scraping process, so that solves a mystery there.

In [ ]:
meets_df[meets_df['meet_id'].isin([27977, 26843, 24822, 21326, 20001, 19660, 20016])]

# Drop any row that has a meet_id that of a row with a missing team_id
meets_df = meets_df[~meets_df['meet_id'].isin([27977, 26843, 24822, 21326, 20001, 19660, 20016])]

# Make sure there's no more NAs in the team_id column
meets_df[meets_df['team_id'].isna()]

In [ ]:
# Change the team_id column to an integer
meets_df['team_id'] = meets_df['team_id'].astype(int)

# Preview the meets_df
meets_df.head()

In [ ]:
%sql DROP TABLE meets;

In [ ]:
%%sql --alias gymternet

CREATE TABLE meets (
    meet_id INTEGER,
    team_id INTEGER,
    meet_year INTEGER
    meet_date DATE,
    meet_url TEXT,
    PRIMARY KEY (meet_id, team_id),
    FOREIGN KEY (team_id) REFERENCES teams (team_id)
);


In [ ]:
meets_df.to_sql('meets', con=engine, if_exists='append', index=False)